In [1]:
import pandas as pd
import numpy as np
import boto3
import configparser
import psycopg2

# Leemos archivo de configuración y credenciales

In [26]:
config = configparser.ConfigParser()
config.read('config_P.cfg')

['config_P.cfg']

In [25]:
aws_rds_conn = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                            aws_secret_access_key=config.get('IAM', 'SECRET_KEY'),
                            region_name='us-east-2')

### Verificamo instancias disponibles para el usuario

In [5]:
rds_instances_ids = []

aws_response = aws_rds_conn.describe_db_instances()

for item in aws_response['DBInstances']:
    rds_instances_ids.append(item['DBInstanceIdentifier'])
    
print(f"Instancias disponibles: {rds_instances_ids}")

Instancias Disponibles: []


### Creamos instancia de base de datos en AWS - Postgres

In [28]:
try:
    response = aws_rds_conn.create_db_instance(
                            DBInstanceIdentifier=config.get('ECOMERCE', 'DB_INSTANCE_ID'),
                            DBName=config.get('ECOMERCE', 'DB_NAME'),
                            MasterUsername=config.get('ECOMERCE', 'DB_USERNAME'),
                            MasterUserPassword=config.get('ECOMERCE', 'DB_PASSWORD'),
                            Port=int(config.get('ECOMERCE', 'DB_PORT')),
                            DBInstanceClass='db.t3.micro',
                            Engine='postgres',
                            PubliclyAccessible=True,
                            AllocatedStorage=20,
                            VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')]
                        )
    print(response)
except aws_rds_conn.exceptions.DBInstanceAlreadyExistsFault:
    print('La instancia ya existe')
except Exception as ex:
    print('Error!!!', ex)

{'DBInstance': {'DBInstanceIdentifier': 'ecomercedb', 'DBInstanceClass': 'db.t3.micro', 'Engine': 'postgres', 'DBInstanceStatus': 'creating', 'MasterUsername': 'ecomercedb', 'DBName': 'ecomerce', 'AllocatedStorage': 20, 'PreferredBackupWindow': '07:03-07:33', 'BackupRetentionPeriod': 1, 'DBSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0c9b1d192d18c4475', 'Status': 'active'}], 'DBParameterGroups': [{'DBParameterGroupName': 'default.postgres16', 'ParameterApplyStatus': 'in-sync'}], 'DBSubnetGroup': {'DBSubnetGroupName': 'default', 'DBSubnetGroupDescription': 'default', 'VpcId': 'vpc-012c32f382ec5723b', 'SubnetGroupStatus': 'Complete', 'Subnets': [{'SubnetIdentifier': 'subnet-0c54fb3d2eb2385cb', 'SubnetAvailabilityZone': {'Name': 'us-east-2c'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-0325c0440844b2e08', 'SubnetAvailabilityZone': {'Name': 'us-east-2b'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-

### Obtenemos el hostname de la instancia

In [31]:
try: 
    instance = aws_rds_conn.describe_db_instances(DBInstanceIdentifier=config.get('ECOMERCE','DB_INSTANCE_ID'))
    RDS_HOSTNAME = instance.get('DBInstances')[0].get('Endpoint').get('Address')
    print(RDS_HOSTNAME)
except Exception as ex:
    print('Error!!!',ex)

ecomercedb.ctiwauocyj1r.us-east-2.rds.amazonaws.com


### Nos conectamos a la base de datos y creamos las tablas

In [66]:
import sql_queries_P

In [ ]:
def insertData2SQL(data_dict, table_name, driver):
    df_data = pd.DataFrame.from_records(data_dict)
    try:
        response = df_data.to_sql(table_name, driver, index=False, if_exists='append')
        print(f'Se han insertado {response} nuevos registros')
    except Exception as ex:
        print('Error!!!', ex)

### Insertamos datos

In [ ]:
data_categoria = [
    {'idcategoria': 1001, 'nombre': 'Furniture', 'descripcion': 'productos como muebles para oficina y casa', 'estado': 'Disponible'},
    {'idcategoria': 1002, 'nombre': 'Toy', 'descripcion': 'conjunto de productos para diversion', 'estado': 'Disponible'}
    {'idcategoria': 1003, 'nombre': 'Health', 'descripcion': 'productos utilizados para la salud', 'estado': 'Disponible'}
    {'idcategoria': 2106, 'nombre': 'Car', 'descripcion': 'productos para vehiculos', 'estado': 'Disponible'}
    {'idcategoria': 2201, 'nombre': 'Electronics', 'descripcion': 'productos electronicos de todo tipo', 'estado': 'Disponible'}
]

#Insertamos los datos en la tabla categoria
insertData2SQL(data_categoria, 'categoria', driver)

In [ ]:
data_rol = 